# 01 - Working with the Azure OpenAI API directly

In this lab, we will perform a couple of simple calls to the Azure OpenAI API.
- The first call will allow us to find out which Model Deployments are available for the Azure OpenAI API.
- The second call will send a prompt to the Azure OpenAI API.

This will also prove that everything is setup correctly and working for the rest of the labs.

## Setup

First, we need to retrieve values from the `.env` file which we will use to make calls to the Azure OpenAI API.

In [1]:
import json
import requests
import os
from dotenv import load_dotenv

# Load environment variables
if load_dotenv():
    print("Found OpenAI API Base Endpoint: " + os.getenv("OPENAI_API_BASE"))
else: 
    print("OpenAI API Base Endpoint not found. Have you configured the .env file?")
    
API_KEY = os.getenv("OPENAI_API_KEY")
# This version of the API is needed to properly retrieve the list of model deployments.
API_VERSION = "2023-03-15-preview"
RESOURCE_ENDPOINT = os.getenv("OPENAI_API_BASE")

Found OpenAI API Base Endpoint: https://openaieastus1121.openai.azure.com/


Next, we'll set the endpoint that we're going to call. This endpoint will return a list of available model deployments.

In [2]:
url = RESOURCE_ENDPOINT + "/openai/deployments?api-version=" + API_VERSION

print (url)

https://openaieastus1121.openai.azure.com//openai/deployments?api-version=2023-03-15-preview


Above you will see the full URL that we are going to be calling.

## Get the list of available Model Deployments

Now let's call that Azure OpenAI endpoint and see what it returns. We pass the API key in the HTTP header.

In [3]:
r = requests.get(url, headers={"api-key": API_KEY})

print(r.text)

{
  "data": [
    {
      "scale_settings": {
        "scale_type": "standard"
      },
      "model": "gpt-4",
      "owner": "organization-owner",
      "id": "gpt44",
      "status": "succeeded",
      "created_at": 1704712756,
      "updated_at": 1704800088,
      "object": "deployment"
    },
    {
      "scale_settings": {
        "scale_type": "standard"
      },
      "model": "gpt-35-turbo",
      "owner": "organization-owner",
      "id": "gpt-35-turbo",
      "status": "succeeded",
      "created_at": 1704796105,
      "updated_at": 1704796105,
      "object": "deployment"
    },
    {
      "scale_settings": {
        "scale_type": "standard"
      },
      "model": "text-embedding-ada-002",
      "owner": "organization-owner",
      "id": "text-embedding-ada-002",
      "status": "succeeded",
      "created_at": 1704796359,
      "updated_at": 1704796359,
      "object": "deployment"
    },
    {
      "scale_settings": {
        "scale_type": "standard"
      },
      "mo

Above, there should now be a JSON formatted list of model deployments. The output will contain one or more sections similar to the following

```
    {
      "scale_settings": {
        "scale_type": "standard"
      },
      "model": "gpt-35-turbo",
      "owner": "organization-owner",
      "id": "gpt35turbo",
      "status": "succeeded",
      "created_at": 1684150536,
      "updated_at": 1684150536,
      "object": "deployment"
    }
```

In the output, you'll see the **Model Name** as the `model` value and the **Deployment Name** as the `id` value. We'll be using an `id` value for the next section.

## Send a prompt to Azure OpenAI using the API

Next, let's call the Azure OpenAI API with a prompt. To do this, we'll need the `id` of our completion model deployment.

We've already setup this value in the `.env` file, but you should see a matching value in the list that was output above.

In [4]:
# Grab API Version in the .env file.
API_VERSION = os.getenv("OPENAI_API_VERSION")
DEPLOYMENT_ID = os.getenv("AZURE_OPENAI_COMPLETION_DEPLOYMENT_NAME")

As before, we'll construct a URL to call. This time, we'll also be creating a payload.

In [5]:
url = RESOURCE_ENDPOINT + "/openai/deployments/" + DEPLOYMENT_ID + "/chat/completions?api-version=" + API_VERSION

print(url)

https://openaieastus1121.openai.azure.com//openai/deployments/gpt-35-turbo/chat/completions?api-version=2023-05-15


Again, you will see the full URL that we are going to call. This URL will use the specified deployment to call the **chat completions** API.

Next, we will call the Azure OpenAI API using the URL above. Just like last time, we pass the API key in the HTTP header. We also send a JSON formatted body as part of the request which contains the *prompt* that we want to use to get a response from the OpenAI model. In this case, our prompt is "Once upon a time", which should cause the model to complete the prompt by generating a story.

In [6]:
r = requests.post(url, headers={"api-key": API_KEY}, json={"messages":[{"role": "assistant", "content": "Once upon a time "}]})

print(json.dumps(r.json(), indent=2))

{
  "id": "chatcmpl-8fMSyplNkmXEZAnzqImhbyYjvIZIY",
  "object": "chat.completion",
  "created": 1704868500,
  "model": "gpt-35-turbo",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "in a small village nestled at the foot of a misty mountain, there was a young girl named Ella. Ella had a wild imagination and a thirst for adventure that often got her into trouble.\n\nOne day, Ella was exploring the outskirts of the village when she stumbled upon an old, decrepit cottage hidden within the dense forest. Intrigued, she cautiously pushed open the creaky door and stepped inside.\n\nTo her surprise, the cottage was filled with an eclectic collection of ancient artifacts and peculiar objects. Ella's eyes widened with excitement as she took in the sight before her. Dust-covered books lined the bookshelves, and peculiar trinkets adorned every surface.\n\nAs Ella made her way through the cottage, she stumble

As before, the result of the API call will be JSON data similar to the below example.

```
{
  "id": "chatcmpl-7wVxE2LNxaY6ZoxcWAmhiyrqaLZgf",
  "object": "chat.completion",
  "created": 1694180212,
  "model": "gpt-35-turbo",
  "choices": [
    {
      "index": 0,
      "finish_reason": "stop",
      "message": {
        "role": "assistant",
        "content": ", there was a magical kingdom called Tildor ... )))"
      }
    }
  ],
  "usage": {
    "completion_tokens": 366,
    "prompt_tokens": 13,
    "total_tokens": 379
  }
}
```

Here's some more information about some of the data contained in that response.

Key | Description
--- | ---
`model` | The model that was used to generate the response to the prompt
`content` | This is the response that was generated by the OpenAI model. Note that the response example above was edited to remove most of the output to make it easier to read
`finish_reason` | This is the reason that the model stopped generating the response. In this case, `stop` indicates the model determined it had fully completed the response without hitting any limits
`completion_tokens` | The number of tokens that were used in generating the response
`prompt_tokens` | The number of tokens that were consumed by the prompt
`total_tokens` | The total number of tokens that were consumed by the request (`prompt_tokens` + `completion_tokens`)

## Summary

In this lab, we used the Azure OpenAI API directly to query information about our instance of the Azure OpenAI service and to send a prompt to an OpenAI model.

## Up Next

In the next lab, we will look at using the OpenAI SDK to work with the Azure OpenAI service.

## Next Section

📣 [OpenAI Packages/Libraries](../02-OpenAIPackages/openai.ipynb)